In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)

In [ ]:
start_date = "2020-01-01"
end_date = "2020-12-31"
data_collection = "SENTINEL-3"
aoi = "POLYGON((-10.0 44.0, -10.0 35.5, 4.0 35.5, 4.0 44.0, -10.0 44.0))"
product_type = "OL_2_LRR___"
username = "1111@1111.com"
password = "11111"
download_dir = "D:/Yuxin/Data/S3_Ref"


top = 50  
skip = 0 

all_products = pd.DataFrame()

while True:

    query_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter=Collection/Name eq '{data_collection}' "
        f"and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') "
        f"and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' "
        f"and att/OData.CSC.StringAttribute/Value eq '{product_type}') "
        f"and ContentDate/Start gt {start_date}T00:00:00.000Z "
        f"and ContentDate/Start lt {end_date}T23:59:59.999Z"
        f"&$orderby=ContentDate/Start desc"
        f"&$top={top}&$skip={skip}"
    )
    
    response = requests.get(query_url)
    json_data = response.json()

    # Check if the returned data has 'value' and contains entries
    if 'value' not in json_data or len(json_data['value']) == 0:
        print("No more products found.")
        break 
    
    # Convert the current batch of data into a DataFrame and append it to the total data
    batch_df = pd.DataFrame.from_dict(json_data['value'])
    all_products = pd.concat([all_products, batch_df], ignore_index=True)
    
    # Print the number of products obtained in the current batch
    print(f"Retrieved {len(batch_df)} products, skipping {skip}.")
    
    # Update paging parameters
    skip += top

# All product information finally obtained
print(f"Total products retrieved: {len(all_products)}")
print(all_products[['Name', 'Id', 'ContentDate']].head())

In [ ]:
# Function to get a new Keycloak token
def get_keycloak(username: str, password: str) -> str:
    """Fetch a new Keycloak access token."""
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]

keycloak_token = get_keycloak(username, password)
session = requests.Session()
session.headers.update({'Authorization': f'Bearer {keycloak_token}'})

# Define your download directory
os.makedirs(download_dir, exist_ok=True)  # Ensure the directory exists

# Function to download a product
def download_product(product_id, product_name, retries=5, retry_delay=30):
    """Download a single product and handle 503 errors."""
    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    attempt = 0
    while attempt < retries:
        try:
            # Handle redirects
            response = session.get(url, allow_redirects=False)
            while response.status_code in (301, 302, 303, 307):
                url = response.headers.get('Location')
                if not url:
                    print(f"Error: Redirect without Location header for product {product_name}")
                    return
                response = session.get(url, allow_redirects=False)

            # Final download
            file_response = session.get(url, allow_redirects=True)
            if file_response.status_code == 200:
                file_path = os.path.join(download_dir, f"{product_name}.zip")
                with open(file_path, "wb") as p:
                    p.write(file_response.content)
                print(f"Downloaded: {product_name}.zip")
                return
            elif file_response.status_code == 401:
                # Token expired, re-authenticate and retry
                print("Token expired. Obtaining a new token...")
                global keycloak_token
                keycloak_token = get_keycloak(username, password)
                session.headers.update({'Authorization': f'Bearer {keycloak_token}'})
                continue  # Retry immediately after refreshing token
            elif file_response.status_code == 503:
                # Service unavailable, wait and retry
                attempt += 1
                print(f"Service unavailable (503). Retrying {attempt}/{retries} in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print(f"Failed to download {product_name}. HTTP Status: {file_response.status_code}")
                return
        except Exception as e:
            print(f"Error downloading {product_name}: {e}")
            return
    print(f"Failed to download {product_name} after {retries} attempts.")

In [ ]:
# target_product_name = "S3A_OL_2_LRR____20190816T093325_20190816T101738_20190817T143105_2653_048_136______LN1_O_NT_002.SEN3"
# try:
#     start_index = all_products[all_products['Name'] == target_product_name].index[0]
#     print(f"Found target product at index: {start_index}")
# except IndexError:
#     print(f"Product with ID {target_product_id} not found in all_products.")
#     exit()

#for index in range(start_index, len(all_products)):
for index in range(0, len(all_products)):
    row = all_products.iloc[index]
    product_id = row['Id']
    product_name = row['Name']
    
    print(f"Downloading product: {product_name} (ID: {product_id})")
    download_product(product_id, product_name)
    time.sleep(1)  # Avoid rate limiting due to excessive requests

print("All downloads complete.")